In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
root = '../characters_parser/data/'

In [3]:
ydf1 = pd.read_csv(root + 'you_anime_characters_refs.csv', sep='\t')
ydf1['anime_url'] = ydf1.page_link.apply(lambda s: s.split('/')[-1])

ydf2 = pd.read_csv(root + 'you_anime_refs.csv', sep='\t')
ydf2['anime_url'] = ydf2.link.apply(lambda s: s.split('/')[-1])

ydf = pd.merge(ydf1, ydf2, on='anime_url')
ydf = ydf['title name'.split()]

ydf.title = ydf.title.apply(str.strip)
ydf.name = ydf.name.apply(str.strip)

ydf.head(3)

,title,name
0,Атака титанов,Эрен Йегер
1,Атака титанов,Микаса Аккерман
2,Атака титанов,Армин Арлерт


In [8]:
names = ydf.name.drop_duplicates().tolist()

In [9]:
len(names)

63065

In [11]:
names[:5]

['Эрен Йегер', 'Микаса Аккерман', 'Армин Арлерт', 'Леви', 'Саша Блауз']

# Read data

In [16]:
from datasets.reader import read_conll
from random import choice

In [35]:
sents = read_conll('./data/comments/raw.txt')
with_character = lambda sent: any(map(lambda token: token[-1] == 'PER', sent))

sents_characters = list(filter(with_character, sents))
len(sents_characters) / len(sents)

0.11923411662315056

# Augment

In [36]:
def get_rand_character(characters):
    char = choice(characters)
    if len(char.split()) > 1:
        char = choice(char.split())
    return char

In [37]:
from copy import deepcopy

In [74]:
augmented_sents_characters = []
AUGMENTATION_ROUNDS = 30

for i in range(AUGMENTATION_ROUNDS):
    for sent in sents_characters:
        sent = deepcopy(sent)
        for i, token in enumerate(sent):
            if token[-1] == 'PER':
                token = list(token)
                token[0] = get_rand_character(names)
                sent[i] = tuple(token)
        augmented_sents_characters.append(sent)

In [75]:
len(augmented_sents_characters)

4110

# Mix data with augmented and save

In [76]:
from random import shuffle

In [77]:
mixed = augmented_sents_characters + sents
shuffle(mixed)

In [78]:
token_to_str = lambda token: '%s _ _ %s' % token
sent_to_str_tokens = lambda sent: '\n'.join(list(map(token_to_str, sent)))

mixed = list(map(sent_to_str_tokens, mixed))
with open('./data/augmented_30_comments/raw.txt', 'w') as f:
    f.write('\n\n'.join(mixed))